In [1]:
# =============================================================
# 🧩 Ensemble Setup & Dataloader
# =============================================================
from pathlib import Path
import torch, pandas as pd, numpy as np
from torch.utils.data import DataLoader
from PIL import Image

# --- Directories ---
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = PROJECT_ROOT / "input_local"
CKPT_DIR = PROJECT_ROOT / "output" / "checkpoints"

device = torch.device("cuda" if torch.cuda.is_available()
                      else ("mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available()
                            else "cpu"))
print(f"🧠 Device: {device}")

# --- Transforms ---
try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    valid_tfms = A.Compose([
        A.Resize(518, 518),
        A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
        ToTensorV2(),
    ])
except ImportError:
    from torchvision import transforms
    valid_tfms = transforms.Compose([
        transforms.Resize((518,518)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
    ])

# --- Dataset ---
class ImageOnlyDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, img_root, transform):
        self.df = pd.read_csv(csv_path)
        self.img_root = Path(img_root)
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(self.img_root / row["image_path"]).convert("RGB")
        import numpy as np
        if "albumentations" in str(type(self.transform)):
            img = np.array(img)
            img = self.transform(image=img)["image"]
        else:
            img = self.transform(img)
        return img, row.get("sample_id", f"test_{idx}")

test_ds = ImageOnlyDataset(DATA_DIR / "test.csv", DATA_DIR, valid_tfms)
test_dl = DataLoader(test_ds, batch_size=8, shuffle=False)
print(f"✅ Test set: {len(test_ds)} samples")


🧠 Device: mps
✅ Test set: 5 samples


In [2]:
# =============================================================
# 🧠 Load Multiple Models & Generate Safe Predictions
# =============================================================
import torch.nn as nn
from timm import create_model

class ImageRegressor(nn.Module):
    def __init__(self, name, num_outputs=5):
        super().__init__()
        self.backbone = create_model(name, pretrained=False, num_classes=0)
        in_features = getattr(self.backbone, "embed_dim", None) or getattr(self.backbone, "num_features", 1024)
        hidden = 768 if "vit" in name else 512
        self.head = nn.Sequential(
            nn.Linear(in_features, hidden), nn.ReLU(), nn.Linear(hidden, num_outputs)
        )
    def forward(self, x): return self.head(self.backbone(x))

def safe_inverse_log(x):
    if np.percentile(np.abs(x), 90) < 15 and x.max() < 30:
        y = np.expm1(x)
        if np.percentile(y, 99) < 1e6:
            return np.clip(y, 0, None)
    return np.clip(x, 0, None)

def infer_with_tta(model, imgs):
    v = [imgs,
         torch.flip(imgs,[2]), torch.flip(imgs,[3]),
         torch.rot90(imgs,1,[2,3]), torch.rot90(imgs,2,[2,3]), torch.rot90(imgs,3,[2,3])]
    preds = [model(x) for x in v]
    return torch.stack(preds).mean(0)

# --- Models to ensemble ---
MODELS = [
    ("vit_base_patch14_dinov2.lvd142m", CKPT_DIR / "vit_base_patch14_dinov2.lvd142m_best.pth"),
    ("efficientnet_b0", CKPT_DIR / "efficientnet_b0_best.pth"),
    ("efficientnet_b3", CKPT_DIR / "efficientnet_b3_best.pth"),
    ("effnet_b0_fusion", CKPT_DIR / "effnet_b0_fusion_best.pth"),
]

target_cols = ["Dry_Clover_g","Dry_Dead_g","Dry_Green_g","Dry_Total_g","GDM_g"]
all_preds = []

for name, path in MODELS:
    if not path.exists(): 
        print(f"⚠️ Missing: {path.name}"); continue
    print(f"🔹 {name}")
    m = ImageRegressor(name).to(device)
    m.load_state_dict(torch.load(path, map_location=device))
    m.eval()
    preds = []
    with torch.no_grad():
        for imgs, _ in test_dl:
            imgs = imgs.to(device)
            out = infer_with_tta(m, imgs).cpu().numpy()
            preds.append(safe_inverse_log(out))
    all_preds.append(np.concatenate(preds))

ensemble_preds = np.mean(all_preds, axis=0)
print(f"✅ Ensemble preds shape: {ensemble_preds.shape}")


🔹 vit_base_patch14_dinov2.lvd142m


RuntimeError: Error(s) in loading state_dict for ImageRegressor:
	Missing key(s) in state_dict: "head.2.weight", "head.2.bias". 
	Unexpected key(s) in state_dict: "head.3.weight", "head.3.bias". 

In [ ]:
# =============================================================
# 📈 Cell 3 — Save Ensemble Submission + Optional Local Evaluation
# =============================================================
import pandas as pd
import numpy as np
from pathlib import Path

# -----------------------------
# 🔹 Prepare for submission
# -----------------------------
sub_df = preds_df.melt(id_vars="image_path", var_name="target_name", value_name="target")

# Use same mapping logic as Kaggle requires
if "sample_id" not in test_df.columns:
    test_df["sample_id"] = test_df["image_path"] + "__" + test_df["target_name"]

final_sub = (
    test_df[["sample_id", "image_path", "target_name"]]
    .merge(sub_df, on=["image_path", "target_name"], how="left")
    .drop_duplicates(subset=["sample_id"])
)

# -----------------------------
# 🔹 Save to output directory
# -----------------------------
OUT_DIR = CKPT_DIR.parent / "submissions"
OUT_DIR.mkdir(parents=True, exist_ok=True)
sub_path = OUT_DIR / "ensemble_submission.csv"

final_sub[["sample_id", "target"]].to_csv(sub_path, index=False)
print(f"💾 Saved ensemble submission → {sub_path.resolve()}")
print(f"📦 Submission shape: {final_sub.shape}")

# -----------------------------
# 🔹 Optional: local ground-truth evaluation (if available)
# -----------------------------
gt_path = DATA_DIR / "test_ground_truth.csv"
if gt_path.exists():
    gt = pd.read_csv(gt_path)
    merged = final_sub.merge(gt, on="sample_id", suffixes=("_pred", "_true"))
    rmse = np.sqrt(np.mean((merged["target_pred"] - merged["target_true"])**2))
    mae  = np.mean(np.abs(merged["target_pred"] - merged["target_true"]))
    print(f"📊 Local RMSE: {rmse:.3f} | MAE: {mae:.3f}")
else:
    print("ℹ️ No local ground truth found — skipping evaluation.")

print("🏁 Ensemble submission ready for Kaggle upload.")
